### Квантует DeepLabV3 MobilenetV3

Стартуем с трейнлупа, который нам выдали pytorch

Датасет COCO, https://cocodataset.org/#download 
Качаем train2017 и val2017

Можно использовать [сабсет](https://drive.google.com/file/d/1qdtAbK-iOsgJZxjbBva0pw2Vi5penjPc/view?usp=sharing) трейна на 20000, но тогда заранее залезте в класс датасета, и добавте работу с пропущенными картинками

Баллы: 20 баллов Static Quantization + 20 баллов Quantization Aware Training + 10 баллов Distillation

In [ ]:
import datetime
import os
import pickle
import time
from copy import deepcopy
from pathlib import Path

import torch
import torch.utils.data
from torch import nn
from torch.ao.quantization.quantize_fx import convert_fx
from torch.ao.quantization.quantize_fx import fuse_fx
from torch.optim.lr_scheduler import PolynomialLR
from torchvision.models.segmentation import DeepLabV3_MobileNet_V3_Large_Weights, deeplabv3_mobilenet_v3_large
from tqdm import tqdm

import utils
from quantization_utils.fake_quantization import fake_quantization
from quantization_utils.static_quantization import quantize_static
from train import evaluate
from train import get_dataset
from train import train_one_epoch

In [ ]:
# Вытащил дефолтные аргументы, чтобы не упражняться с argparse в ноутбуке
with Path('./torch_default_args.pickle').open('rb') as file:
    args = pickle.load(file)

In [ ]:
# Подобирайте под ваше железо
args.data_path = '/home/d.chudakov/datasets/coco/'
args.epochs = 1
args.batch_size = 32
args.workers = 8

In [ ]:
args

### Сначала просто валидация обычной сетки, прям на гпу

In [ ]:
model = deeplabv3_mobilenet_v3_large(weights=DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT)
model.eval();

In [ ]:
if args.output_dir:
    utils.mkdir(args.output_dir)

utils.init_distributed_mode(args)

device = torch.device(args.device)

dataset_test, num_classes = get_dataset(args, is_train=False)

test_sampler = torch.utils.data.SequentialSampler(dataset_test)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=32, sampler=test_sampler, num_workers=args.workers, collate_fn=utils.collate_fn
)

In [ ]:
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
model.cuda()
confmat = evaluate(model, data_loader_test, device=device, num_classes=num_classes)
print(confmat)

### Заквантуем статические сетку, посмотрим на точность и скорость

In [ ]:
# Квантуем
# Делаем fuse, делаем quantize_static и quantize_utils (посмотрите что там с кодом)
# Можно покрутить параметр num_batches, чтобы посмотреть сколько нужно данных на калибровку

In [ ]:
# Замерим скорость квантованной модели на CPU
# Не забываем, от размера батча будет зависить буст!

In [ ]:
# Замеряем с бачтом 1, буста нет
# Замеряем с батчом 32, буст есть
# Мораль, latency != throughput. В сетке всегда есть накладные расходы, кроме перемалывания матричек

In [ ]:
# Замерим скорость оригинальной модели на CPU
# У меня на intel i9 при батчсайзе 32 получился x2 буст

In [ ]:
# Посчитаем метрики квантованной модели
# У меня была просадка где-то до 56 IoU
q_model.cpu()
confmat = evaluate(q_model, data_loader_test, device='cpu', num_classes=num_classes)
print(confmat)

### Делаем Quantization Aware Training. Используем готовый трейнплуп от pytorch

In [ ]:
# Делаем фейк квантование, идём смотреть quantization_utils

### Тут берём из train.py скрипт main() и вытаскиваем трейн луп

1. Не забыть провалидировать модель fake quant до qat
2. Не забыть провалидировать модель после обучения
3. Конвертировать модель из fake quant в обычный quant
4. Проверить точность и скорость модели
Должно хватить пары эпох, lr надо будет покрутить.
Цель минимум это поднять точность конвертированной QAT модели (будет у вас QAT до обучения 55, станет 56, молодцы!)

In [ ]:
# Подсказка, учить надо в train и на GPU
qat_model.cuda();

In [ ]:
# Инференс делаем на cpu, предварительно конвертируя модельку на CPU
qat_model.cpu()
int_qat_model = convert_fx(qat_model)

In [ ]:
# Точность модели fake quant и квантованной после конвертации будут разные
# Так и должно быть, всё таки мы эмулировали квантование.

### Балуемся с дистилляцией
Врываемся в train.py и добавляем туда дистилляцию, просто по последнему слою (до софтмакса, на логитах) делаем стягивание по MSE

Цель поднять точность и ускорить сходимость.

Балуемся с весами обычного и distill лосса.

Можно вообще выкинуть classification loss и смоделировать ситуацию когда вам не выдали лейблов (жиза)